# Quick Example for AutoDiff w/ Mass

Just computes generalized forces for a simple body.

We create a simple `MultibodyPlant` (`MultibodyPlant_[float]`). This means you *could*
parse it from an URDF / SDFormat file.
We then convert the system to `AutoDiffXd`, and set the parameters we desire. In this case,
we want $\frac{\partial{f}}{\partial{m}}$, where $f$ is just some arbitrary expression.

In our case, we choose $f$ to be generalized forces at the default / home configuration.
Also in this case, we choose $m \in \mathbb{R}^2$ just to show how to choose gradients for
independent values.

For more information, please see:
- [Underactuated: System Identification](http://underactuated.csail.mit.edu/sysid.html) - at present, this only presents the symbolic approach for `MultibodyPlant`.
- [nbviewer: `drake/tutorials/dynamic_systems.ipynb`](https://nbviewer.jupyter.org/github/RobotLocomotion/drake/blob/v0.30.0/tutorials/dynamical_systems.ipynb)
- [nbviewer: `drake/tutorials/mathematical_program_multibody_plant.ipynb`](https://nbviewer.jupyter.org/github/RobotLocomotion/drake/blob/v0.30.0/tutorials/mathematical_program_multibody_plant.ipynb)

In [1]:
import numpy as np
from pydrake.all import (
    MultibodyPlant_,
    MultibodyPlant,
    AutoDiffXd,
    SpatialInertia,
    UnitInertia,
)

In [2]:
plant = MultibodyPlant(time_step=0.0)
body1 = plant.AddRigidBody(
    "body1",
    M_BBo_B=SpatialInertia(
        mass=2.0,
        p_PScm_E=[0, 0, 0],
        # N.B. This value may not be physically "useful". I just guessed.
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
body2 = plant.AddRigidBody(
    "body2",
    M_BBo_B=SpatialInertia(
        mass=0.5,
        p_PScm_E=[0, 0, 0],
        # N.B. This value may not be physically "useful". I just guessed.
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
plant.Finalize()

plant_ad = plant.ToScalarType[AutoDiffXd]()
body1_ad = plant_ad.get_body(body1.index())
body2_ad = plant_ad.get_body(body2.index())
context_ad = plant_ad.CreateDefaultContext()

Here we now populate the parameters we wish to take the gradient with respect to.

For forward-mode automatic differentiation, we must ensure that specify our gradients
according to our desired independent variables. In this case, we want $m_1$ and $m_2$
to be independent, so we ensure their gradients are distinct unit vectors.

If we wanted, we could set more parameters, but we stick to just mass for simplicity.

In [3]:
m1 = AutoDiffXd(2.0, [1.0, 0.0])
body1_ad.SetMass(context_ad, m1)

m2 = AutoDiffXd(0.5, [0.0, 1.0])
body2_ad.SetMass(context_ad, m2)

The generalized force, in the z-translation direction for each body $i$, should just be $(-m_i \cdot g)$ with derivative $(-g)$.

In [4]:
def get_z_component(plant, body, v):
    assert body.is_floating()
    # N.B. This method returns position w.r.t. *state* [q; v]. We only have v (or vdot).
    x_start = body.floating_velocities_start()
    # Floating-base velocity dofs are organized as [angular velocity; translation velocity].
    v_start = x_start - plant.num_positions()
    nv_pose = 6
    rxyz_txyz = v[v_start:v_start + nv_pose]
    txyz = rxyz_txyz[-3:]
    z = txyz[2]
    return z

In [5]:
@np.vectorize
def ad_to_string(x):
    return f"AutoDiffXd({x.value()}, derivatives={x.derivatives()})"

In [6]:
tau_g = plant_ad.CalcGravityGeneralizedForces(context_ad)
tau_g_z1 = get_z_component(plant_ad, body1_ad, tau_g)
tau_g_z2 = get_z_component(plant_ad, body2_ad, tau_g)
print(ad_to_string(tau_g_z1))
print(ad_to_string(tau_g_z2))

AutoDiffXd(-19.62, derivatives=[-9.81 -0.  ])
AutoDiffXd(-4.905, derivatives=[-0.   -9.81])
